-sandbox
#### PROYECTO DATAHACK

In [0]:
# Cargando librerias necesarias
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
import pyspark.sql.functions as sf

# Leyendo world_happiness_report.csv, informacion hasta la gestion 2020
data_whr = (spark
            .read
            .format("csv")
            .option("delimiter", ",")
            .option("header", "true")
            .option("inferSchema", "true")
            .load("/FileStore/tables/world_happiness_report.csv")
            .withColumnRenamed("Logged GDP per capita","Logged_GDP_per_capita")
            .withColumnRenamed("Country name", "Country_name")
           )

# Leyendo world_happiness_report_2021.csv, informacion de la gestion 2021 
data_whr_2021 = (spark
                 .read
                 .format("csv")
                 .option("delimiter", ",")
                 .option("header", "true")
                 .option("inferSchema", "true")
                 .load("/FileStore/tables/world_happiness_report_2021.csv"))

#Renombrando columnas y quitando los espacion en los nombres de las columnas
data_whr_2021 = (data_whr_2021
                 .withColumnRenamed("Country name", "Country_name")
                 .withColumnRenamed("Regional indicator", "Regional_indicator")
                 .withColumnRenamed("Ladder score","Ladder_score")
                 .withColumnRenamed("Standard error of ladder score","se_ladder_score")
                 .withColumnRenamed("Logged GDP per capita","Logged_GDP_per_capita")
                 .withColumnRenamed("Social support","Social_support")
                 .withColumnRenamed("Healthy life expectancy","Healthy_life_expectancy")
                 .withColumnRenamed("Freedom to make life choices","Freedom_to_make_life_choices")
                 .withColumnRenamed("Perceptions of corruption","Perceptions_of_corruption")
                 .withColumnRenamed("Ladder score in Dystopia","Ladder_score_in_Dystopia")
                 .withColumnRenamed("Explained by: Log GDP per capita","Explained_by_Log_GDP_per_capita")
                 .withColumnRenamed("Explained by: Social support","Explained_by_Social_support")
                 .withColumnRenamed("Explained by: Healthy life expectancy","Explained_by_Healthy_life_expectancy")
                 .withColumnRenamed("Explained by: Freedom to make life choices","Explained_by_Freedom_to_make_life_choices")
                 .withColumnRenamed("Explained by: Generosity","Explained_by_Generosity")
                 .withColumnRenamed("Explained by: Perceptions of corruption","Explained_by_Perceptions_of_corruption")
                 .withColumnRenamed("Dystopia + residual","Dystopia_mas_residual")
                )
# mostrando los 5 primeras filas
display(data_whr_2021.limit(5))

Country_name,Regional_indicator,Ladder_score,se_ladder_score,upperwhisker,lowerwhisker,Logged_GDP_per_capita,Social_support,Healthy_life_expectancy,Freedom_to_make_life_choices,Generosity,Perceptions_of_corruption,Ladder_score_in_Dystopia,Explained_by_Log_GDP_per_capita,Explained_by_Social_support,Explained_by_Healthy_life_expectancy,Explained_by_Freedom_to_make_life_choices,Explained_by_Generosity,Explained_by_Perceptions_of_corruption,Dystopia_mas_residual
Finland,Western Europe,7.842,0.032,7.904,7.78,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
Denmark,Western Europe,7.62,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.03,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
Switzerland,Western Europe,7.571,0.036,7.643,7.5,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
Iceland,Western Europe,7.554,0.059,7.67,7.438,10.878,0.983,73.0,0.955,0.16,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.17,2.967
Netherlands,Western Europe,7.464,0.027,7.518,7.41,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


-sandbox
##### PREGUNTA 1. ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Life Leadder” mayor número más feliz es el país)

In [0]:
# Primera forma para mostrar el pais con mayor indice de felicidad
import pyspark.sql.functions as sf
happy_country_21 = (data_whr_2021
                    .select("Country_name", "Ladder_score")
                    .groupBy("Country_name")
                    .agg(sf.max("Ladder_score").alias("Indice_Felicidad"))
                    .orderBy("Indice_Felicidad", ascending=False)
                   )
display(happy_country_21.limit(5))
# Para el 2021 el pais con el indice mayor de felicidad es Finlandia.

Country_name,Indice_Felicidad
Finland,7.842
Denmark,7.62
Switzerland,7.571
Iceland,7.554
Netherlands,7.464


In [0]:
# Segunda forma para mostrar el pais con myor indice de felicidad, con una vista temporal y mediante SQL
countryHappy2  = data_whr_2021.createOrReplaceTempView("happyCountry")
df2 = spark.sql("select Country_name, sum(Ladder_score) as Indice_Felicidad from happyCountry group by Country_name order by Indice_Felicidad desc")
display(df2.limit(5))
# El pais con mayor indice de felicidad es Finlandia, mostrando los datos en orden ascendente.

Country_name,Indice_Felicidad
Finland,7.842
Denmark,7.62
Switzerland,7.571
Iceland,7.554
Netherlands,7.464


-sandbox
##### PREGUNTA 2. ¿Cuál es el país más “feliz” del 2021 por continente según la data?

In [0]:
# Inicialmente se creo un DATAFRAME happy_continent con el indice maximo de felicidad por continente
happy_continent = spark.sql("select Regional_indicator, "+ 
                            "max(Ladder_score) as Indice_Felicidad " +
                            "from happyCountry " + 
                            "group by Regional_indicator" 
                           )
# Se creo una vista temporal con los resultados 
happy_continent.createOrReplaceTempView("tabla_continentes")

# luego se hizo un inner join solo para ir a obtener el nombre de la ciudad
df_continent = spark.sql("select tbl1.Regional_indicator" +
                         ",tbl2.Country_name " + 
                         ",tbl1.Indice_Felicidad " + 
                         "from tabla_continentes tbl1 INNER JOIN happyCountry tbl2 ON tbl1.Regional_indicator = tbl2.Regional_indicator AND tbl1.Indice_Felicidad = tbl2.Ladder_score ")
display(df_continent)
# Y este es el resultado del pais mas feliz por continente del 2021

Regional_indicator,Country_name,Indice_Felicidad
Western Europe,Finland,7.842
North America and ANZ,New Zealand,7.277
Middle East and North Africa,Israel,7.157
Latin America and Caribbean,Costa Rica,7.069
Central and Eastern Europe,Czech Republic,6.965
East Asia,Taiwan Province of China,6.584
Southeast Asia,Singapore,6.377
Commonwealth of Independent States,Uzbekistan,6.179
Sub-Saharan Africa,Mauritius,6.049
South Asia,Nepal,5.269


-sandbox
##### PREGUNTA 3. ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?

In [0]:
# Data Frame con todas las gestiones hasta la 2020
data_whr_all_years = (data_whr
                     .withColumnRenamed("Country name", "Country_name")
                     .withColumnRenamed("Life Ladder","Life_Ladder")
                     .select(sf.col("year"), sf.col("Country_name"), sf.col("Life_Ladder")) 
                      )

# Como sabemos que el pais con mayor indice de felicidad de la gestion 2021 es finlandia lo añadimos al dataframe de todas las gestiones
df2021 = (happy_country_21
          .filter(sf.col("Country_name")=="Finland") 
          .select(sf.col("Country_name"), sf.col("Indice_Felicidad").alias("Life_Ladder"))         
          .withColumn("year", sf.lit(2021))
         )

# ordenamos columnas para el union
df2021 = (df2021.select(sf.col("year"), sf.col("Country_name"), sf.col("Life_Ladder")))

# unimos los dos dataFrames
df_complete = df2021.union(data_whr_all_years)
# creamos una columna rownumber para dar un orden o marcar a los maximos con el numero 1
windowSpec  = Window.partitionBy(sf.col("year")).orderBy(sf.col("score").desc())
df = (df_complete
      .select("Country_name","year", "Life_Ladder")
      .groupBy("year","Country_name")
      .agg(sf.max("Life_Ladder").alias("score"))
      .withColumn("row_number",row_number().over(windowSpec))
     )
display(df.limit(5))

year,Country_name,score,row_number
2005,Denmark,8.019,1
2005,Netherlands,7.464,2
2005,Canada,7.418,3
2005,Sweden,7.376,4
2005,Australia,7.341,5


In [0]:
# Paises con mayor indice de felicidad por gestion
paises_primer_lugar = (df.filter(sf.col("row_number")==1).orderBy(sf.col("year")))
display(paises_primer_lugar)

year,Country_name,score,row_number
2005,Denmark,8.019,1
2006,Finland,7.672,1
2007,Denmark,7.834,1
2008,Denmark,7.971,1
2009,Denmark,7.683,1
2010,Denmark,7.771,1
2011,Denmark,7.788,1
2012,Switzerland,7.776,1
2013,Canada,7.594,1
2014,Denmark,7.508,1


In [0]:
# agrupando por pais para saber quien se repitio mas veces
pais_mas_veces = (paises_primer_lugar
                 .select(sf.col("Country_name"))
                 .groupBy(sf.col("Country_name"))
                 .count()
                 .sort('count', ascending=False) 
                 )
display(pais_mas_veces)

Country_name,count
Finland,7
Denmark,7
Norway,1
Switzerland,1
Canada,1


-sandbox
#### Según la tabla de resultados existen dos países que obtuvieron el primer lugar mas veces entre las gestiones 2005 a la 2021, los países son Finlandia y Dinamarca.